# snowflake

> Working with Snowflake

In [ ]:
#| default_exp snowflake

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from triggerkit import util
from triggerkit import actions
import snowflake.connector
import os, re
import pandas as pd
from snowflake.connector.connection import SnowflakeConnection
from snowflake.connector.cursor import SnowflakeCursor
from typing import Dict, List, Callable, Optional, Any, Union

2025-05-15 23:44:24,667 - numexpr.utils - INFO - NumExpr defaulting to 10 threads.


In [ ]:
#| export

util.VIEW_REGISTRY: Dict[str, Dict[str, Any]] = {}

def register_view(name: str, query: str, metadata: Optional[Dict[str, Any]] = None):
    """
    Register a Snowflake view with the system.
    
    Args:
        name: Name of the view
        query: SQL query or view name to fetch datay
        metadata: Additional metadata about the view
        
    Returns:
        None
    """
    util.VIEW_REGISTRY[name] = {
        'query': query,
        'metadata': metadata or {}
    }
    util.logger.info(f"Registered view: {name}")

In [ ]:
#| export

@actions.register('Register Views','Registers all views in data with VIEW_REGISTRY.')
def register_views(data: List[Dict[str, Any]], context, config):
    """
    Registers all views in data with VIEW_REGISTRY.
    """
    for view in data:
        register_view(
            view.get('name'),
            view.get('query'),
            {'description': view.get('description')}
            )

2025-05-15 23:44:24,906 - triggerkit - INFO - Registered action: Register Views


In [ ]:
#| export 

util.snowflake_connection = None

def connect(sf_config: Dict[str, Any]) -> SnowflakeConnection:
    """
    Establish connection to Snowflake.
    
    Args:
        sf_config: Dictionary containing Snowflake connection parameters
        
    Returns:
        Snowflake connection object
    """
    # Check for environment variables for sensitive data
    password = os.environ.get('SNOWFLAKE_PASSWORD') or sf_config.get('password')
    
    try:
        connection = snowflake.connector.connect(
            user=sf_config.get('user'),
            password=password,
            account=sf_config.get('account'),
            warehouse=sf_config.get('warehouse'),
            database=sf_config.get('database'),
            schema=sf_config.get('schema'),
            role=sf_config.get('role', 'ACCOUNTADMIN')
            #,session_parameters={'QUERY_TAG': 'TRIGGERKIT'}
        )
        util.logger.info("Successfully connected to Snowflake")
        return connection
    except Exception as e:
        util.logger.error(f"Failed to connect to Snowflake: {str(e)}")
        raise

In [ ]:
#| export

def query(sql_query: str) -> List[Dict[str, Any]]:
    """
    Execute a query and return results.
    
    Args:
        sql_query: SQL query to execute
        
    Returns:
        List of result rows as dictionaries
    """
    if not hasattr(util, 'snowflake_connection') or  hasattr(util, 'snowflake_connection') and util.snowflake_connection == None:
        util.snowflake_connection = connect(util.SNOWFLAKE_CONFIG)
    try:
        cursor: SnowflakeCursor = util.snowflake_connection.cursor()
        cursor.execute(sql_query)
        column_names = [col[0] for col in cursor.description]
        
        # Convert results to list of dictionaries
        results = []
        for row in cursor:
            results.append(dict(zip(column_names, row)))
        cursor.close()
        return results
    except Exception as e:
        print('Query:', sql_query)
        util.logger.error(f"Failed to execute query: {str(e)}")
        raise Exception(e)

def get_view_data(view_name: str) -> List[Dict[str, Any]]:
    """
    Execute a query for a registered view and return results.
    
    Args:
        view_name: Name of the registered view
        
    Returns:
        List of result rows as dictionaries
    """
    if view_name not in util.VIEW_REGISTRY:
        raise ValueError(f"View '{view_name}' is not registered")
    
    view_config = util.VIEW_REGISTRY[view_name]
    sql_query = view_config['query']
    
    try:
        results = query(sql_query)
        util.logger.info(f"Successfully executed query for view '{view_name}', returned {len(results)} rows")
        return results
    except Exception as e:
        util.logger.error(f"Failed to execute query for view '{view_name}': {str(e)}")
        raise

In [ ]:
#| export

def extract_view_sql(ddl: str) -> str:
    """
    Extracts the SQL query from a Snowflake view definition.
    
    Args:
        ddl: Snowflake view definition
    
    Returns:
        Extracted SQL query
    """

    if 'CREATE' not in ddl.upper(): return ddl
    
    ddl = ddl.strip().replace(';','')

    match = re.search(r'\bAS\b\s*(\()?[\s\n]*', ddl, flags=re.IGNORECASE)
    if match:
        start = match.end()
        sql = ddl[start:].strip()
    else:
        return ddl

    if sql.startswith('(') and sql.endswith(')'):
        try:
            balance = 0
            for c in sql:
                if c == '(':
                    balance += 1
                elif c == ')':
                    balance -= 1
                if balance < 0:
                    break
            if balance == 0:
                sql = sql[1:-1].strip()
        except Exception:
            pass

    return sql

@actions.register('Get DDL and Register Views','Given view names, it will retrieve the views\' definition and register the views.')
def get_ddl_and_register(data, context, config):
    """
    For each view in data, retrieve the DDL and register the view. 
    """
    database = util.SNOWFLAKE_CONFIG.get('database')
    check_schema = util.SNOWFLAKE_CONFIG.get('check_schema')

    results = []

    for view_info in data:
        view_name = view_info['TABLE_NAME']

        ddl_query = f"""
            SELECT GET_DDL('VIEW', '{database}.{check_schema}.{view_name}') AS full_ddl
        """
        view_ddl_result = query(ddl_query)

        if view_ddl_result:
            raw_ddl = view_ddl_result[0]['FULL_DDL']
            cleaned_sql = extract_view_sql(raw_ddl)

            results.append({
                'name': view_name,
                'query': cleaned_sql
            })

    register_views(results)


2025-05-15 23:44:24,919 - triggerkit - INFO - Registered action: Get DDL and Register Views


In [ ]:
#| exporti

def _check_schema(config: Dict[str, Any]):
    """
    Retrieves all views in 'check_schema' and extracts only the SELECT/WITH logic from the view definition.
    """
    if not config.get('snowflake') or not config['snowflake'].get('check_schema'):
        return None

    database = config['snowflake'].get('database')
    check_schema = config['snowflake'].get('check_schema')

    # Query to get all of the views defined in the check_schema
    views_query = f"""
        SELECT table_name
        FROM {database}.INFORMATION_SCHEMA.VIEWS
        WHERE table_schema = '{check_schema}'
     """
    
    # Query to get all of the views that contain TK_CONFIG or CONFIG columns
    views_with_config_cols_query = f"""
        SELECT DISTINCT table_name
        FROM {database}.INFORMATION_SCHEMA.COLUMNS
        WHERE table_schema = '{check_schema}'
          AND column_name ILIKE ANY ('TK_CONFIG', 'CONFIG')
          AND data_type ILIKE ANY ('VARIANT', 'OBJECT')
    """

    try:
        view_names_list = query(views_query)

        return view_names_list, views_query, views_with_config_cols_query

    except Exception as e:
        print(f"Error executing query: {e}")
        return None

In [ ]:
#| export

def init_view_registry(config: Dict[str, Any]):
    """
    Starts View Registry by including all locally defined views and including all views
    located in the the schema defined in the configuration file.
    
    Args:
        config: Dictionary containing view configurations
    
    Returns:
        None
    """

    if config.get('views'): register_views(config.get('views'))
    
    schema_views, check_schema_query, views_with_config_cols_query = _check_schema(config)

    if schema_views: 
        get_ddl_and_register(schema_views)
        register_view('tk_check_schema', check_schema_query)
        register_view('tk_cofigured_views', views_with_config_cols_query)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()